## Final Project - NLP-Driven Ingredient Health and Dietary Restriction Analysis

*Name: Laura Obermaier*

*Stevens ID: 20027358*

#### Imports

In [1]:
import re
import csv
import requests
import spacy
import pubchempy as pcp
from rapidfuzz import process, fuzz
from collections import defaultdict
from spacy.lang.en import English
import tiktoken
from googleapiclient.discovery import build
from dotenv import load_dotenv
import os
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM
from rouge_score import rouge_scorer
import numpy as np
from bert_score import score as bert_score
import inflect
from Bio import Entrez

#### Environment & Global Setup

In [2]:
# Load environment variables from .env file
load_dotenv()

# Retrieve API keys from environment
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
#USDA_API_KEY = os.getenv("USDA_API_KEY")
#APILEAGUE_API_KEY = os.getenv("APILEAGUE_API_KEY")

inflect_engine = inflect.engine()

# Increase CSV field size limit to support large text fields
csv.field_size_limit(2**20)

# Load spaCy's English NLP pipeline
nlp = spacy.load("en_core_web_sm")

# Initialize a basic sentencizer pipeline for sentence splitting
nlp_sentencizer = English()
nlp_sentencizer.add_pipe("sentencizer")
global_alias_set = set()

In [3]:
#used for web scraping
keyword_config = {
    "benefits": [
        "benefit", "supports", "improves", "boosts", "enhances", "aids", "reduces", "prevents",
        "protects", "promotes", "stimulates", "strengthens", "aiding", "improving", "healing",
        "facilitates", "enhancing", "balances", "restores", "ameliorates", "treats", "alleviates",
        "relieves", "contributes to", "beneficial", "positive", "advantageous", "favorable",
        "healthy", "wellness", "nutrient", "nutritional", "immune", "well-being", "absorption",
        "energy", "fitness", "cognitive", "focus", "clarity", "relief", "anti-inflammatory",
        "cramps", "gut", "probiotic", "enzyme", "alkaline", "acidic", "bloating", "constipation"
    ],
    "concerns": [
        "risk", "toxic", "harm", "adverse", "cause", "increased", "linked to", "danger",
        "poisonous", "unsafe", "negatively", "exacerbates", "side effect",
        "carcinogenic", "neurotoxic", "hepatotoxic", "irritation", "may lead to", "trigger",
        "overdoes", "reaction", "symptom", "pain", "toxins"
    ],
    "dietary restrictions": [
        "allergy", "intolerance", "sensitivity", "restricted", "avoid", "not suitable",
        "contraindicated", "dietary restriction", "religious restriction", "vegan", "vegetarian",
        "halal", "haram", "gluten", "lactose", "kosher", "FODMAP", "contains", "may contain",
        "cross-contamination", "not recommended", "not advised", "not suitable for",
        "not safe for"
    ],
    "neutral impact": [
        "kidney", "liver", "cholesterol", "diabetes", "inflammation", "cancer", "cardiovascular",
        "digestion", "therapy", "treatment", "metabolism", "metabolic", "calories"
    ],
    "health": [
        "health", "wellness", "nutrition", "nutritional", "dietary", "diet", "supplement", "vitamin", "mineral",
        "well-being", "immune", "absorption", "energy", "fitness", "cognitive", "focus",
        "clarity", "relief", "anti-inflammatory", "cramps", "gut", "probiotic", "enzyme", 
    ],
}

# Compose superset for health relevance filtering
keyword_config["all"] = list(set(
    keyword_config["benefits"] +
    keyword_config["concerns"] +
    keyword_config["dietary restrictions"] +
    keyword_config["neutral impact"] +
    keyword_config["health"]
))

#### Model Initialization

In [4]:
# Define the model path for the quantized Mistral-7B instruction-tuned model
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"

# Load the tokenizer with fast tokenization and support for custom code
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    use_fast=True,
    trust_remote_code=True
)

# Load the quantized Mistral-7B model with safetensors and automatic device mapping
model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    use_safetensors=True,
    device_map="auto",
    trust_remote_code=True,
    revision="main"
)

c:\Users\laura\anaconda3\envs\llama3gptq\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO - The layer lm_head is not quantized.


#### Aliases & Ingredient Standardization

In [5]:
# Checks if an alias is relevant by applying several filters based on length, character content, and known undesirable terms
def is_relevant_alias(alias):
    alias_clean = alias.strip().lower()
    if len(alias_clean.split()) > 4:  # Discard if alias has more than 4 words
        return False
    if re.search(r'\d{3,}|\d+%|[^\w\s\-]', alias_clean):  # Discard if contains long numbers, percentages, or symbols
        return False
    if re.search(r'^\d{2,5}-\d{2,5}-\d$', alias_clean):  # Discard barcode-like patterns
        return False
    if re.search(r"\b(usp|grade|reagent|testing|microg|ml|specification|mixture|vetec|methanol|water|preparation)\b", alias_clean):
        return False  # Remove known irrelevant descriptors
    if len(alias_clean) > 40:  # Discard overly long aliases
        return False
    if alias_clean.count(',') > 0 or alias_clean.count('(') > 1:  # Discard if messy formatting
        return False
    if any(keyword in alias_clean for keyword in ['acs', 'usp', 'grade', 'reference', 'powder', 'solution', 'mist']):
        return False  # Filter out chemical/formulation terms
    return True

def pluralize_word(word):
    plural = inflect_engine.plural(word)
    if plural and plural != word:
        print(f"[DEBUG] Pluralized '{word}' → '{plural}'")
        return plural
    return None

def normalize_alias_list(aliases):
    cleaned = set()
    for alias in aliases:
        alias = alias.strip().lower()
        alias = re.sub(r'\s{2,}.*$', '', alias)  # Truncate after 2+ spaces
        cleaned.add(alias)
    return list(cleaned)

def prompt_scientific_name_from_model(ingredient, model, tokenizer):
    prompt = f"Scientific name of food ingredient '{ingredient}' is"
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **input_ids,
        max_new_tokens=15,
        do_sample=False,
        num_beams=3,
        early_stopping=True
    )
    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    answer = generated.replace(prompt, "").strip()
    answer = re.split(r"[^\S ]", answer)[0]  # Splits on any whitespace except space
    # Clean up punctuation
    answer = re.sub(r"[\(\)\[\]\"',.]", "", answer).strip()

    if 1 <= len(answer.split()) <= 4:
        print(f"[Model Fallback] {ingredient} → Scientific: {answer}")
        return answer
    print(f"[Model Fallback ⚠️ Invalid scientific name for '{ingredient}']: {answer}")
    return None

def prompt_common_name_from_model(scientific_name, model, tokenizer):
    prompt = f"Common name of scientific food ingredient '{scientific_name}' is"
    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **input_ids,
        max_new_tokens=15,
        do_sample=False,
        num_beams=3,
        early_stopping=True
    )
    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    answer = generated.replace(prompt, "").strip()
    answer = re.split(r"[^\S ]", answer)[0]  # Splits on any whitespace except space
    # Clean up punctuation
    answer = re.sub(r"[\(\)\[\]\"',.]", "", answer).strip()

    if 1 <= len(answer.split()) <= 5:
        print(f"[Model Fallback] {scientific_name} → Common: {answer}")
        return answer
    print(f"[Model Fallback ⚠️ Invalid common name for '{scientific_name}']: {answer}")
    return None
    
def get_food_aliases(name):
    name = name.lower().strip()
    aliases = set()

    print(f"[DEBUG ALIASES 1] before pluralize of {name}: {aliases}\n")

    # Add plural/singular forms
    plural = pluralize_word(name)
    if plural and plural != name:
        aliases.add(plural)
    if name.endswith('s'):
        singular = name[:-1]
        if singular:
            aliases.add(singular)

    print(f"[DEBUG ALIASES 2] after pluralize & before scientific additions of {name}: {aliases}\n")

    # Step 2: Use LLM prompts for scientific/common name expansion
    if model and tokenizer:
        scientific_name = prompt_scientific_name_from_model(name, model, tokenizer)
        common_name = prompt_common_name_from_model(scientific_name, model, tokenizer)
        if scientific_name:
            aliases.add(scientific_name)
        if common_name:
            aliases.add(common_name)

    print(f"[DEBUG ALIASES 3] after scientific additions of {name}: {aliases}\n")

    return normalize_alias_list([a for a in aliases if is_relevant_alias(a)]) or [name]

def fuzzy_match_alias(name, threshold=90):
    if not global_alias_set:
        print("[Warning] Alias set is empty — did you run seed_aliases_from_open_food_facts?")
        return None
    result = process.extractOne(name, global_alias_set, scorer=fuzz.token_sort_ratio)
    if result is None:
        return None
    match, score, _ = result
    return match if score >= threshold else None

# Checks if a word is phonetically valid by ensuring a reasonable vowel/consonant ratio
def is_phonetically_valid(word):
    word = word.lower()
    if len(word) < 3:
        return False
    vowels = sum(1 for c in word if c in "aeiou")
    consonants = sum(1 for c in word if c.isalpha() and c not in "aeiou")
    if consonants == 0:
        return False
    ratio = vowels / (consonants + vowels)
    return 0.2 <= ratio <= 0.8  # Reject if ratio is too low or high

# Main alias standardization routine for ingredient names
def standardize_ingredient_name(name):
    name = name.lower().strip()
    aliases = get_food_aliases(name)
    print(f"[Query Alias] {name} → All aliases: {aliases}")
    if aliases:
        return name, aliases
    fuzzy = fuzzy_match_alias(name)
    if fuzzy:
        print(f"2. [Query Alias] {name} → Fuzzy-corrected: {fuzzy}")
        return fuzzy, [fuzzy]
    print(f"[Query Alias] {name} → No aliases: {name}")
    return name, [name]

#### External API Queries

In [6]:
# Query PubMed for article summaries related to the given ingredient and aliases
def query_pubmed(ingredient, aliases=None, max_results=100):
    try:
        # Construct query using aliases and health-related keywords
        alias_query = " OR ".join([f'"{a}"' for a in aliases])
        keywords_query = " OR ".join(keyword_config["all"])
        final_query = f"({alias_query}) AND ({keywords_query})"

        # Perform initial search to get PMIDs
        base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        params = {"db": "pubmed", "term": final_query, "retmode": "json", "retmax": max_results}
        ids = requests.get(base_url, params=params).json()["esearchresult"].get("idlist", [])

        summaries = []
        # Use retrieved IDs to fetch article summaries
        for pmid in ids:
            summary_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
            r = requests.get(summary_url, params={"db": "pubmed", "id": pmid, "retmode": "json"}).json()
            result = r["result"].get(pmid)
            if result:
                summaries.append({"title": result.get("title"), "source": result.get("source"), "pubdate": result.get("pubdate")})
        return summaries
    except:
        return []  # Return empty list on any failure

# Check whether an OpenFDA recall entry is relevant to the ingredient
def is_fda_entry_relevant(text, aliases):
    irrelevant_keywords = [
        "recall", "undeclared", "labeling", "distribution",
        "pasteurization", "packaging", "incorrect", "contain"
    ]
    text_lower = text.lower()
    if any(kw in text_lower for kw in irrelevant_keywords):  # Filter out irrelevant recalls
        return False
    return any(alias.lower() in text_lower for alias in aliases)  # Check if any alias is present

# Query OpenFDA for relevant recall entries based on aliases
def query_openfda(ingredient, aliases=None, max_results=100):
    aliases = aliases or [ingredient]
    try:
        query = " OR ".join([f'"{a}"' for a in aliases])
        base_url = "https://api.fda.gov/food/enforcement.json"
        params = {"search": f"product_description:({query})", "limit": max_results}
        r = requests.get(base_url, params=params).json()
        results = r.get("results", [])

        filtered = []
        # Filter results based on relevancy
        for rec in results:
            reason = rec.get("reason_for_recall", "")
            if is_fda_entry_relevant(reason, aliases):
                filtered.append(reason)

        return list(set(filtered))  # Remove duplicates
    except Exception as e:
        print(f"[OpenFDA error for aliases {aliases}]: {e}")
        return []

# Query RxNorm for medical identifiers corresponding to aliases
def query_rxnorm(ingredient, aliases=None, max_results=100):
    aliases = aliases or [ingredient]
    try:
        url = "https://rxnav.nlm.nih.gov/REST/rxcui.json"
        all_ids = set()

        # Query RxNorm for each alias
        for alias in aliases[:max_results]:
            rxcui = requests.get(url, params={"name": alias}).json()
            ids = rxcui.get("idGroup", {}).get("rxnormId", [])
            all_ids.update(ids)

        return list(all_ids)
    except Exception as e:
        print(f"[RxNorm error for aliases {aliases}]: {e}")
        return []

# Query multiple academic sources (EuropePMC and PubMed Central) for relevant articles
def query_academic_health_docs(ingredient, aliases=None, max_results=100):
    
    # Subfunction to query EuropePMC API for relevant abstracts
    def query_europe_pmc(aliases=None, max_results=100):
        try:
            base_url = "https://www.ebi.ac.uk/europepmc/webservices/rest/search"
            if aliases is None:
                aliases = [ingredient]

            # Build query from aliases and health keywords
            alias_query = " OR ".join([f'"{a}"' for a in aliases])
            keywords_query = " OR ".join(keyword_config["all"])
            query = f"({alias_query}) AND ({keywords_query})"

            params = {
                "query": query,
                "format": "json",
                "resultType": "core",
                "sort": "P_PDATE_D",
                "pageSize": max_results
            }

            r = requests.get(base_url, params=params).json()
            results = []
            for record in r.get("resultList", {}).get("result", []):
                abstract = record.get("abstractText")
                if not abstract or len(abstract.strip()) < 100:
                    continue  # Skip short or missing abstracts
                results.append({
                    "title": record.get("title"),
                    "source": record.get("journalTitle"),
                    "pubdate": record.get("firstPublicationDate", record.get("pubYear")),
                    "url": f"https://europepmc.org/article/{record.get('source')}/{record.get('id')}",
                    "full_text": abstract.strip()
                })
            return results
        except Exception as e:
            print(f"[EuropePMC JSON API error for '{ingredient}']: {e}")
            return []

    # Subfunction to query PubMed Central (PMC) for article metadata
    def query_pubmed_central(aliases=None, max_results=100):
        try:
            search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
            if aliases is None:
                aliases = [ingredient]

            alias_query = " OR ".join([f'"{a}"' for a in aliases])
            keywords_query = " OR ".join(keyword_config["all"])
            query = f"({alias_query}) AND ({keywords_query})"

            params = {
                "db": "pmc",
                "term": query,
                "retmode": "json",
                "retmax": max_results,
                "sort": "pub+date"
            }

            r = requests.get(search_url, params=params).json()
            ids = r.get("esearchresult", {}).get("idlist", [])
            summaries = []

            # For each ID, fetch title and construct URL
            for pmid in ids:
                try:
                    summary_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
                    summary_resp = requests.get(summary_url, params={"db": "pmc", "id": pmid, "retmode": "json"}).json()
                    result = summary_resp.get("result", {}).get(pmid)
                    if not result:
                        continue
                    title = result.get("title", "")
                    url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC{pmid}/"
                    summaries.append({"title": title, "url": url, "full_text": title})
                except:
                    continue
            return summaries
        except Exception as e:
            print(f"[PubMedCentral error for '{ingredient}']: {e}")
            return []

    # Combine results from both sources and remove duplicate titles
    pmc_results = query_pubmed_central(aliases)
    europepmc_results = query_europe_pmc(aliases)
    combined = pmc_results + europepmc_results
    seen_titles = set()
    unique_results = []
    for r in combined:
        if r["title"] and r["title"] not in seen_titles:
            unique_results.append(r)
            seen_titles.add(r["title"])
    return unique_results[:max_results]

#### NER & Web Snipped Extraction

In [7]:
# Extract named entities from the text with contextual filtering
def extract_entities(text, aliases=None, health_keywords=None):
    doc = nlp(text)

    # Labels and words to ignore (e.g., dates, money, etc.)
    invalid_labels = {
        "CARDINAL", "DATE", "ORDINAL", "PERCENT", "LANGUAGE", "TIME", "QUANTITY", "MONEY", "NORP", "EVENT"
    }
    forbidden_words = {
        "recall", "product", "distribution", "ingredient", "label", "cookie", "brownie", "package",
        "expiration", "sell", "pasteurization"
    }

    spans = []
    for ent in doc.ents:
        span_text = ent.text.strip()
        span_clean = span_text.lower()
        label = ent.label_

        # Reclassify any alias that was misclassified as a PERSON
        if label == "PERSON" and aliases and span_clean in aliases:
            print(f"[NER Correction] '{span_text}' was labeled as PERSON, relabeling as INGREDIENT")
            label = "INGREDIENT"

        # Skip irrelevant or short entities
        if label in invalid_labels:
            continue
        if len(span_text) < 3:
            continue
        if any(word in span_clean for word in forbidden_words):
            continue

        # Score based on keyword context within ±50 characters
        context_window = text[max(0, ent.start_char - 50):ent.end_char + 50].lower()
        context_score = sum(1 for kw in (health_keywords or []) if kw in context_window)

        spans.append((ent.start_char, ent.end_char, label, span_text, context_score))

    # Deduplicate overlapping entities by keeping the most relevant one
    merged = []
    spans.sort(key=lambda x: (x[0], -(x[1]-x[0])))
    for start, end, label, text_span, score in spans:
        if merged and start < merged[-1][1]:
            prev = merged[-1]
            if score > prev[4]:  # keep more relevant span
                merged[-1] = (start, end, label, text_span, score)
        else:
            merged.append((start, end, label, text_span, score))

    # Return final list of entity strings and their labels
    return [(text[start:end], label) for start, end, label, _, _ in merged]

# Use Google CSE to retrieve health-related web snippets for ingredient aliases
def search_web_snippets(ingredient, aliases=None, num_results=100, api_key=None, cse_id=None):
    if not api_key or not cse_id:
        raise ValueError("Google API key and CSE ID are required.")

    if aliases is None:
        aliases = [ingredient]

    # Create alias query and divide keywords into chunks for separate searches
    alias_query = " OR ".join([f'"{a}"' for a in aliases])
    keywords = keyword_config["all"]
    keyword_groups = [keywords[i:i+20] for i in range(0, len(keywords), 20)]

    all_snippets = []

    for group in keyword_groups:
        keyword_query = " OR ".join([f'"{kw}"' for kw in group])
        query = f"({alias_query}) AND ({keyword_query})"

        try:
            service = build("customsearch", "v1", developerKey=api_key)
            res = service.cse().list(
                q=query,
                cx=cse_id,
                num=min(num_results, 10),  # Google API only allows 10 results per call
                sort="date"
            ).execute()
            items = res.get("items", [])
            snippets = [item.get("snippet", "") for item in items if item.get("snippet")]
            print(f"[Google CSE] Retrieved {len(snippets)} snippets for keyword batch: {group[:3]}...")
            all_snippets.extend(snippets)
        except Exception as e:
            print(f"[Google CSE Error for query]: {query}\n{e}")
            continue

    return list(set(all_snippets))  # Remove duplicates from all retrieved snippets

# High-level wrapper to retrieve and extract health-relevant entities from the web
def semantic_scrape_summary(ingredient, api_key=None, cse_id=None):
    fallback_attempts = 0
    all_snippets = search_web_snippets(ingredient, aliases=[ingredient], api_key=api_key, cse_id=cse_id)

    # Fallback to sanitized version if no results found
    if not all_snippets:
        fallback_attempts += 1
        fallback_term = re.sub(r'[^\w\s]', '', ingredient)
        print(f"[Fallback] Trying sanitized alias: '{fallback_term}'")
        all_snippets = search_web_snippets(fallback_term, aliases=[fallback_term], api_key=api_key, cse_id=cse_id)

    if not all_snippets:
        print(f"[Fallback] Google CSE returned no snippets even after fallback. Skipping.")
        return []

    all_ents = []
    # Apply NER to all snippets and collect entities
    for i, snippet in enumerate(all_snippets):
        if not snippet.strip():
            continue
        ents = extract_entities(snippet, aliases=global_alias_set, health_keywords=keyword_config["all"])
        all_ents.extend(ents)

    return list(set(all_ents))  # Return deduplicated set of entities

#### Ingredient List Preprocessing

In [8]:
# Retrieve all health-related information for a given ingredient using multiple trusted APIs
def get_all_health_info(ingredient, aliases=None):
    aliases = aliases or [ingredient]
    return {
        "PubMed": query_pubmed(ingredient, aliases),                     # Get scientific article metadata
        "OpenFDA": query_openfda(ingredient, aliases),                   # Get FDA recall or warning data
        "RxNorm": query_rxnorm(ingredient, aliases),                     # Get RxNorm medical identifiers
        "Academic_Articles": query_academic_health_docs(ingredient, aliases)  # Get abstracts/full-text from EuropePMC and PubMed Central
    }

# Main preprocessing pipeline to clean a raw ingredient list and retrieve health info per ingredient
def preprocess_ingredient_list_with_health(text):
    raw_ingredients = re.split(r'[\,\n;/••]+', text)  # Split list by common delimiters
    processed = []
    seen_terms = set()

    for raw in raw_ingredients:
        raw = raw.strip()
        if not raw:
            continue
        
        # Skip terms unlikely to be valid ingredients based on vowel/consonant ratio
        if not is_phonetically_valid(raw):
            print(f"[!] Skipping '{raw}' — unlikely to be a valid ingredient (too short or invalid vowel/consonant pattern)")
            continue
        # Normalize ingredient name and expand with aliases
        standard, aliases = standardize_ingredient_name(raw)
        aliases = list(set([standard] + aliases))  # Deduplicate and include base term

        # Rank aliases by fuzzy similarity to original input
        ranked_aliases = sorted(
            aliases,
            key=lambda x: fuzz.token_sort_ratio(raw.lower(), x.lower()),
            reverse=True
        )
        selected_aliases = ranked_aliases[:5]  # Use top 5 most similar aliases

        # Initialize containers for different info sources
        combined_health_info = {
            "PubMed": [],
            "OpenFDA": [],
            "RxNorm": [],
            "NER_Snippets": [],
            "Academic_Articles": []
        }

        found_any_data = False

        # Retrieve health info for each selected alias
        for term in selected_aliases:
            if term.lower() in seen_terms:
                continue
            seen_terms.add(term.lower())

            # Query APIs and semantic web scraping
            api_info = get_all_health_info(term, selected_aliases)
            ner_info = semantic_scrape_summary(term, api_key=GOOGLE_API_KEY, cse_id=GOOGLE_CSE_ID)

            # Check if any relevant data was found
            if any(api_info[k] for k in api_info) or ner_info:
                found_any_data = True

            # Merge results from all APIs and NER scraping into the ingredient's health_info
            for k in combined_health_info:
                if k == "NER_Snippets":
                    combined_health_info[k].extend([i for i in ner_info if i not in combined_health_info[k]])
                else:
                    combined_health_info[k].extend([i for i in api_info[k] if i not in combined_health_info[k]])

        # Warn if nothing was retrieved for this ingredient
        if not found_any_data:
            print(f"⚠️ No data found for '{raw}'. Did you spell this correctly or is it too obscure?")

        # Save the processed entry
        processed.append({
            "standard": standard,
            "aliases": aliases,
            "health_info": combined_health_info
        })

    return processed  # List of dictionaries, each representing a processed ingredient

#### Summarization & Postprocessing

In [9]:
# --- Summarization Functions ---
def generate_summary(text, category, ingredient_name, model, tokenizer):
    # Choose prompt based on category type
    if category == "health effects":
        instruction = f"From the following scientific findings, summarize how {ingredient_name} affects human health. Include both health benefits and health concerns clearly labeled. Limit to 5 bullet points."
    elif category == "dietary restrictions":
        instruction = f"From the following scientific findings, summarize how {ingredient_name} affects human dietary restrictions (such as allergies, intolerances, and religious restrictions). Limit to 5 bullet points."
    else:
        raise ValueError("Invalid category")

    prompt = instruction + "\n\n" + text  # Combine instruction with findings

    # Tokenize and generate model output
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()  # Decode model output

    # Post-processing: extract bullet points from output
    lines = summary.splitlines()
    lines = lines[1:]  # Remove instruction line if included
    bullets = []
    for i, line in enumerate(lines, start=1):
        stripped = line.lstrip()
        if not stripped:
            continue
        if stripped.startswith("-"):  # Clean dash-based bullets
            content = stripped[1:].strip()
        else:
            content = stripped

        if not re.match(r"^\d+\.\s", content):  # Add numbering if missing
            content = f"{i}. {content}"
        content = f"        {content}"  # Indent bullets
        bullets.append(content)

    bullets[0] = f"\n{bullets[0]}"  # Separate first bullet with newline

    # Deduplicate bullet points using fuzzy matching
    unique_bullets = []
    for bullet in bullets:
        if all(fuzz.ratio(bullet, b) < 90 for b in unique_bullets):
            unique_bullets.append(bullet)

    # Keep only well-formed bullet points with sentence-ending punctuation
    cleaned_bullets = []
    for bullet in unique_bullets:
        if bullet.strip()[-1] in {".", "!", "?"}:
            cleaned_bullets.append(bullet)

    # Return top 5 cleaned bullets or fallback to raw summary
    return "\n".join(cleaned_bullets[:5]) if cleaned_bullets else summary

def deduplicate_sentences(sentences, threshold=92):
    """Deduplicate semantically similar sentences based on fuzzy matching."""
    unique = []
    for s in sentences:
        if all(fuzz.ratio(s, u) < threshold for u in unique):
            unique.append(s)
    return unique

def truncate_to_token_limit(text, tokenizer, max_tokens=800):
    # Truncate text to fit within the specified token budget
    tokens = tokenizer.encode(text, truncation=True, max_length=max_tokens)
    return tokenizer.decode(tokens, skip_special_tokens=True)

def normalize_aliases(sentences, aliases, ingredient_name):
    """Replaces all aliases in the sentences with the main ingredient name."""
    normalized = []
    for sentence in sentences:
        for alias in sorted(aliases, key=len, reverse=True):  # Replace longest aliases first
            pattern = re.compile(rf"\b{re.escape(alias)}\b", flags=re.IGNORECASE)
            sentence = pattern.sub(ingredient_name, sentence)
        normalized.append(sentence)
    return normalized

#### Sentence Extraction & Filtering

In [10]:
def clean_and_chunk_sentences(text, health_keywords=None, max_sentences=20):
    # Tokenize input text into sentences
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    
    # Filter sentences based on health-related keywords if provided
    if health_keywords:
        sentences = [s for s in sentences if any(kw in s.lower() for kw in health_keywords)]
    
    # Group sentences into chunks of max_sentences each
    chunks = []
    for i in range(0, len(sentences), max_sentences):
        chunks.append(" ".join(sentences[i:i+max_sentences]))
    
    return chunks

def filter_sentences(sentences, task):
    # Filter sentences based on keyword config for the given task category
    if task in keyword_config:
        keywords = keyword_config[task]
        return [s for s in sentences if any(word in s.lower() for word in keywords)]
    return sentences

def is_probably_human_study(title, abstract, ingredient_name=None, aliases=None):
    # Define terms indicating animal studies
    non_human_terms = [
        "mouse", "mice", "rat", "rabbits", "pigs", "cattle", "chicken", "fish", "gobies", 
        "canine", "murine", "insect", "drosophila", "dog", "animal model", "rodent", "zebrafish"
    ]
    
    # Define terms indicating plant-focused studies
    plant_terms = [
        "chlorophyll", "photosynthesis", "agronomic", "germination", "arabidopsis", 
        "transpiration", "mesophyll", "stomata", "cultivar", "xylem", "phloem", 
        "leaf senescence", "plant tissue culture", "callus induction", 
        "agriculture", "horticulture", "cultivation", "field trial", 
        "harvest", "agroecosystem", "rhizosphere", "seed germination", 
        "soil nutrient", "nitrogen fixation"
    ]

    combined_text = (title + " " + abstract).lower()

    # Check if ingredient name or its aliases contain plant-related words
    ingredient_terms = [ingredient_name.lower()] if ingredient_name else []
    if aliases:
        ingredient_terms.extend(a.lower() for a in aliases)
    
    plant_terms_in_ingredient = any(
        plant_term in term for plant_term in plant_terms for term in ingredient_terms
    )

    # If ingredient isn't plant-related, filter both animal and plant studies
    combined_filter_terms = non_human_terms + plant_terms if not plant_terms_in_ingredient else non_human_terms

    # Return True if no filter terms are found → likely human study
    return not any(term in combined_text for term in combined_filter_terms)

def extract_relevant_sentences(text, aliases=None, title=None):
    # Keywords that signal important result/conclusion sentences
    outcome_terms = [
        "result", "conclusion", "finding", "found", "significant", "associated",
        "led to", "revealed", "observed", "showed", "demonstrated", "indicated",
        "suggested", "implied", "noted", "reported", "discovered", "established",
    ]
    # Phrases to ignore to skip boilerplate content
    skip_phrases = {"this study aims", "background", "introduction", "study design", "was conducted", "was performed"}
    keywords = set(keyword_config["all"] + outcome_terms)

    aliases = [a.lower() for a in (aliases or [])]
    reference_text = ((title or "") + " " + text).lower()

    # Check if any alias appears in the document
    if not any(alias in reference_text for alias in aliases):
        return []

    # Extract only the relevant, non-boilerplate sentences
    return [
        sent.text for sent in nlp(text).sents
        if any(kw in sent.text.lower() for kw in keywords)
        and not any(phrase in sent.text.lower() for phrase in skip_phrases)
    ]

def enrich_with_health_summaries_v2(results, model, tokenizer):
    for entry in results:
        ingredient_name = entry['standard']
        aliases = entry.get("aliases", [ingredient_name])
        articles = entry["health_info"].get("Academic_Articles", [])
        print(f"\n🧪 Processing ingredient: {ingredient_name}")
        print(f"→ Total academic articles found: {len(articles)}")

        if not articles:
            entry["health_summary"] = {"note": "No academic articles available."}
            continue

        # Filter out articles that are not about humans (or ingredient-specific plants)
        human_articles = [
            a for a in articles 
            if is_probably_human_study(a.get("title", ""), a.get("full_text", ""), ingredient_name=ingredient_name, aliases=aliases)
        ]
        print(f"→ Human-relevant articles retained: {len(human_articles)}")
        print(f"→ Articles rejected for non-human focus: {len(articles) - len(human_articles)}")

        if not human_articles:
            entry["health_summary"] = {"note": "No human-relevant articles found."}
            continue

        entry["health_summary"] = {}

        # Generate summaries separately for health effects and dietary restrictions
        for category in ["health effects", "dietary restrictions"]:
            category_keywords = (
                keyword_config["benefits"] + keyword_config["concerns"]
                if category == "health effects"
                else keyword_config["dietary restrictions"]
            )

            category_summaries = []
            total_relevant_sentences = 0

            for article in human_articles:
                raw_text = article.get("full_text", "")
                title = article.get("title", "")
                if not raw_text:
                    continue

                # Extract and filter relevant sentences
                relevant_sentences = extract_relevant_sentences(raw_text, aliases=aliases, title=title)
                total_relevant_sentences += len(relevant_sentences)

                filtered_sentences = [
                    s for s in relevant_sentences if any(kw in s.lower() for kw in category_keywords)
                ]
                filtered_sentences = deduplicate_sentences(filtered_sentences)
                filtered_sentences = normalize_aliases(filtered_sentences, aliases, ingredient_name)

                if not filtered_sentences:
                    continue

                # Combine sentences, truncate to token limit, and generate summary
                combined_text = " ".join(filtered_sentences)
                truncated_input = truncate_to_token_limit(combined_text, tokenizer)

                summary = generate_summary(
                    truncated_input, category, ingredient_name, model, tokenizer
                )
                category_summaries.append(summary)

            # Use fallback summarization if no summaries were produced
            if not category_summaries:
                fallback_chunks = []
                for article in human_articles:
                    title = article.get("title", "").strip()
                    source = article.get("source", "").strip()
                    if len(title) > 20:
                        snippet = ""  # omit title text from summary
                        if source:
                            snippet += f" | Source: {source}"
                        fallback_chunks.append(snippet)
                fallback_chunks = normalize_aliases(fallback_chunks, aliases, ingredient_name)
                if fallback_chunks:
                    combined_text = " ".join(fallback_chunks)
                    truncated_input = truncate_to_token_limit(combined_text, tokenizer)
                    fallback_summary = generate_summary(
                        truncated_input, category, ingredient_name, model, tokenizer
                    )
                    entry["health_summary"][category] = fallback_summary
                else:
                    entry["health_summary"][category] = f"No relevant {category} information found."
            else:
                entry["health_summary"][category] = "\n".join(list(set(category_summaries)))
    return results

#### Dynamize Fruity Pebbles Protein Powder: Test Run, Output, and Evaluation

##### Test Run

In [11]:
dynamize_fruity_pebbles_ingredients = "Hydrolyzed Whey Protein Isolate, Whey Protein Isolate, Natural Flavors, Artificial Flavors, Gum Arabic, FD&C #3, FD&C Red #40, FD&C Blue#1, FD&C Blue #1, FD&C Yellow#5, Salt, Soy Lecithin, Sucralose, Stevia"
dynamize_fruity_pebbles_results = preprocess_ingredient_list_with_health(dynamize_fruity_pebbles_ingredients)

# Run LLM-based summarization on your enriched ingredient data
dynamize_fruity_pebbles_results = enrich_with_health_summaries_v2(dynamize_fruity_pebbles_results, model, tokenizer)

[DEBUG ALIASES 1] before pluralize of hydrolyzed whey protein isolate: set()

[DEBUG] Pluralized 'hydrolyzed whey protein isolate' → 'hydrolyzed whey protein isolates'
[DEBUG ALIASES 2] after pluralize & before scientific additions of hydrolyzed whey protein isolate: {'hydrolyzed whey protein isolates'}



c:\Users\laura\anaconda3\envs\llama3gptq\lib\site-packages\transformers\integrations\sdpa_attention.py:54: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[Model Fallback ⚠️ Invalid scientific name for 'hydrolyzed whey protein isolate']: : Hydrolyzed whey protein isolate is a type of protein
[Model Fallback ⚠️ Invalid common name for 'None']: not an acceptable value for the common_name field
[DEBUG ALIASES 3] after scientific additions of hydrolyzed whey protein isolate: {'hydrolyzed whey protein isolates'}

[Query Alias] hydrolyzed whey protein isolate → All aliases: ['hydrolyzed whey protein isolates']
[Google CSE] Retrieved 10 snippets for keyword batch: ['healing', 'vegetarian', 'may lead to']...
[Google CSE] Retrieved 10 snippets for keyword batch: ['supports', 'strengthens', 'alkaline']...
[Google CSE] Retrieved 10 snippets for keyword batch: ['danger', 'supplement', 'vitamin']...
[Google CSE] Retrieved 10 snippets for keyword batch: ['focus', 'not suitable', 'religious restriction']...
[Google CSE] Retrieved 10 snippets for keyword batch: ['toxins', 'healthy', 'mineral']...
[Google CSE] Retrieved 10 snippets for keyword batch: ['i

##### Output

In [12]:
# Print trusted API info cleanly
print("TRUSTED API INFO:\n=================")
for entry in dynamize_fruity_pebbles_results:
    print(f"\n🧪 Ingredient: {entry['standard']}")
    print("→ Aliases:", entry["aliases"])

    print("\nTrusted API Info:")
    for source, data in entry["health_info"].items():
        if source == "NER_Snippets":
            continue
        print(f"  • {source}:")
        if isinstance(data, list) and data:
            for item in data:
                if isinstance(item, str):
                    print(f"     - {item}")
                elif isinstance(item, dict):
                    print(f"     - {item.get('title', '')}")
        elif isinstance(data, list):
            print("     - No results")
        else:
            print(f"     - {data}")

# Print the NER Entitiies cleanly
print("\n\nNER ENTITIES:\n=============")
for entry in dynamize_fruity_pebbles_results:
    print(f"\n🧪 Ingredient: {entry['standard']}")
    print("→ Aliases:", entry["aliases"])

    print("\nNER Entities from Web Snippets:")
    if entry["health_info"]["NER_Snippets"]:
        for ent_text, ent_label in entry["health_info"]["NER_Snippets"]:
            print(f"     - {ent_text} ({ent_label})")
    else:
        print("     - No named entities found.")

TRUSTED API INFO:

🧪 Ingredient: hydrolyzed whey protein isolate
→ Aliases: ['hydrolyzed whey protein isolate', 'hydrolyzed whey protein isolates']

Trusted API Info:
  • PubMed:
     - No results
  • OpenFDA:
     - No results
  • RxNorm:
     - No results
  • Academic_Articles:
     - Unraveling the Biological Properties of Whey Peptides and Their Role as Emerging Therapeutics in Immune Tolerance
     - Effects of time-of-day resistance training on muscle strength, hormonal adaptations, and sleep quality during Ramadan fasting
     - Lactiplantibacillus plantarum LM1001 Improves Digestibility of Branched-Chain Amino Acids in Whey Proteins and Promotes Myogenesis in C2C12 Myotubes
     - Proteins and Amino Acids Treated with Atmospheric Plasma Show Significantly Increased Bioavailability in Humans
     - Do the anatomical and physiological properties of a muscle determine its adaptive response to different loading protocols?
     - Four Weeks of Time-Restricted Feeding Combined with R

In [13]:
# Print the results (Summaries) cleanly
print("LLM-GENERATED HEALTH SUMMARIES:\n===============================")
for entry in dynamize_fruity_pebbles_results:
    print(f"\n🧪 Ingredient: {entry['standard']}")
    print("\n💬 LLM-Generated Health Summaries:")
    for category, summary in entry.get("health_summary", {}).items():
        print(f"  [{category.upper()}]: {summary}")
    print("\n")

LLM-GENERATED HEALTH SUMMARIES:

🧪 Ingredient: hydrolyzed whey protein isolate

💬 LLM-Generated Health Summaries:
  [HEALTH EFFECTS]: 
        1. Improved Muscle Protein Synthesis: Hydrolyzed whey protein isolate (WPI) has been shown to enhance muscle protein synthesis, leading to increased muscle mass and strength in both resistance-trained and untrained individuals (Paddon-Jones et al., 2001; Rasmussen et al., 2000).
        2. Faster Digestion and Absorption: The hydrolysis process breaks down the protein into smaller peptides and amino acids, making it easier for the body to digest and absorb quickly, providing a rapid delivery of essential amino acids to the muscles (Tang et al., 1999).
  [DIETARY RESTRICTIONS]: 
        1. Hydrolyzed whey protein isolate (WPI) is derived from cow's milk, making it unsuitable for individuals with lactose intolerance or milk allergies. However, some hydrolyzed WPI products may be lactose-free due to the enzymatic hydrolysis process that breaks down

##### Reference Summaries

In [14]:
# --- Reference summaries --- (using CHATGPT-4 and human evaluation to fine-tune)
dynamize_fruity_pebbles_reference_summaries = {
    "hydrolyzed whey protein isolate-health_effects": (
        "Hydrolyzed whey protein isolate is quickly absorbed by the body and supports rapid muscle recovery after exercise. "
        "It contains essential amino acids that promote muscle growth and repair. "
        "It may help maintain lean body mass and improve athletic performance. "
        "Its hydrolyzed form may be easier to digest than regular whey. "
        "It may also contribute to increased satiety and better metabolic health."
    ),
    "hydrolyzed whey protein isolate-restrictions": (
        "Hydrolyzed whey protein isolate is derived from milk and is not suitable for individuals with milk allergies or strict vegan diets. "
        "Though hydrolyzation reduces lactose, individuals with severe lactose intolerance may still experience discomfort. "
        "It is also unsuitable for those following dairy-free diets for ethical or health reasons."
    ),

    "whey protein isolate-health_effects": (
        "Whey protein isolate promotes muscle growth and supports recovery after exercise. "
        "It is rich in branched-chain amino acids (BCAAs) that are crucial for muscle synthesis. "
        "It may help regulate appetite and support weight management. "
        "It is low in fat and carbohydrates, making it ideal for lean mass gain. "
        "Some studies suggest it supports immune function due to its cysteine content."
    ),
    "whey protein isolate-restrictions": (
        "Whey protein isolate is not suitable for individuals with milk allergies or those following a vegan lifestyle. "
        "Although it is low in lactose, some individuals with lactose intolerance may still experience symptoms. "
        "People with kidney disease should consult a healthcare provider before consuming high-protein supplements."
    ),

    "natural flavors-health_effects": (
        "They generally do not have significant health effects in small amounts. "
        "The specific health impact is hard to assess due to the lack of transparency in flavor composition."
    ),
    "natural flavors-restrictions": (
        "Natural flavors can be derived from animal sources and may not be suitable for vegans or vegetarians. "
        "Individuals with food allergies should exercise caution, as natural flavors may contain allergenic substances. "
        "People following halal or kosher diets may need clarification from manufacturers to determine acceptability."
    ),

    "artificial flavors-health_effects": (
        "They provide no nutritional benefits. "
        "Some consumers prefer to avoid them due to concerns about long-term safety and chemical exposure."
    ),
    "artificial flavors-restrictions": (
        "Artificial flavors are not suitable for individuals seeking clean-label or all-natural products. "
        "Certain artificial flavoring agents may be restricted in specific countries due to regulatory guidelines. "
        "Individuals with chemical sensitivities may prefer to avoid them."
    ),
    "gum arabic-health_effects": (
        "Gum arabic is a natural fiber derived from the acacia tree and used as a stabilizer and emulsifier. "
        "It may support gut health due to its prebiotic properties. "
        "It has a low glycemic index and is considered safe for most people. "
        "Some evidence suggests it may help with cholesterol reduction and appetite regulation."
    ),
    "gum arabic-restrictions": (
        "Gum arabic may cause bloating or mild gastrointestinal discomfort in sensitive individuals. "
        "It is generally well tolerated but should be avoided in high doses by those with irritable bowel syndrome. "
        "Some people may be allergic to it, especially if they have tree pollen allergies."
    ),

    "fd&c #3-health_effects": (
        "It has been linked to thyroid tumors in animal studies, although not definitively in humans. "
        "Some studies suggest it may cause hyperactivity in children. "
        "It provides no nutritional benefit and is banned in certain countries for use in cosmetics."
    ),
    "fd&c #3-restrictions": (
        "Individuals with dye sensitivities or behavioral conditions such as ADHD may wish to avoid it. "
        "It is not suitable for consumers seeking clean-label or additive-free products."
    ),

    "fd&c red #40-health_effects": (
        "It has been associated with allergic reactions, particularly in individuals with aspirin sensitivity. "
        "Some research links it to hyperactivity in children, though findings are mixed. "
    ),
    "fd&c red #40-restrictions": (
        "It is not suitable for people with dye allergies. "
        "Those following strict diets like Feingold may avoid it."
    ),

    "fd&c blue #1-health_effects": (
        "It is generally considered safe but may cause allergic reactions in sensitive individuals. "
        "Some animal studies have shown adverse effects on nerve cells, although evidence in humans is limited."
    ),
    "fd&c blue #1-restrictions": (
        "FD&C Blue 1 may cause hypersensitivity reactions in some individuals. "
        "It is unsuitable for those avoiding synthetic dyes or practicing natural food diets."
    ),

    "fd&c yellow #5-health_effects": (
        "It has been linked to hyperactivity in children and allergic reactions in some individuals. "
        "It provides no nutritional value and is often viewed negatively by clean-eating proponents."
    ),
    "fd&c yellow #5-restrictions": (
        "FD&C Yellow 5 must be declared on food labels in the U.S. due to known allergic reactions. "
        "It is not acceptable in additive-free, organic, or dye-restricted diets."
    ),

    "salt-health_effects": (
        "Salt (sodium chloride) is essential for nerve and muscle function. "
        "However, excessive intake can lead to high blood pressure, heart disease, and stroke. "
        "It can cause water retention and kidney strain when consumed in large amounts. "
        "Moderation is key to balancing its benefits and risks."
    ),
    "salt-restrictions": (
        "Salt must be restricted in low-sodium diets, often prescribed for hypertension or cardiovascular disease. "
        "It may be avoided in renal diets to reduce fluid retention. "
        "Some religious fasting practices also restrict salt intake."
    ),

    "soy lecithin-health_effects": (
        "Soy lecithin is a natural emulsifier that may support cognitive health due to its choline content. "
        "It may help manage cholesterol levels and promote liver health. "
        "Some studies suggest potential antioxidant effects."
    ),
    "soy lecithin-restrictions": (
        "Soy lecithin is derived from soybeans and is not suitable for individuals with soy allergies. "
        "It is also unsuitable for those avoiding genetically modified ingredients, unless labeled non-GMO. "
        "People following soy-free or elimination diets should avoid it."
    ),

    "sucralose-health_effects": (
        "It is generally considered safe but may alter gut microbiota with long-term use. "
        "Some individuals report headaches or bloating after consumption. "
    ),
    "sucralose-restrictions": (
        "Sucralose is not suitable for individuals avoiding artificial sweeteners. "
        "Some people with digestive disorders may prefer to avoid it. "
        "It is not allowed in certain organic food certifications."
    ),

    "stevia-health_effects": (
        "It has no calories and may help manage blood sugar levels. "
        "It is considered safe for most people and may support weight management. "
        "Some studies suggest antioxidant and anti-inflammatory properties."
    ),
    "stevia-restrictions": (
        "Stevia may cause bloating or nausea in some sensitive individuals. "
        "Individuals allergic to ragweed may also react to stevia."
    ),
}


##### Evaluation

In [15]:
# --- Evaluation functions ---

# Set up metric containers
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

rouge1_scores, rougeL_scores = [], [],
bert_precisions, bert_recalls, bert_f1s = [], [], []

# Evaluation loop
for entry in dynamize_fruity_pebbles_results:
    ingredient = entry["standard"].lower()
    for category, summary in entry.get("health_summary", {}).items():
        key = f"{ingredient}-{category.replace(' ', '_')}"
        reference = dynamize_fruity_pebbles_reference_summaries.get(key)

        if not reference or not summary.strip():
            continue

        # Extract generated bullets
        generated_bullets = [
            line.strip()[2:].strip()
            for line in summary.splitlines()
            if re.match(r"^\d+\.", line.strip())
        ]
        if not generated_bullets:
            continue

        # Extract reference sentences
        reference_bullets = [s.strip() for s in reference.split('.') if s.strip()]
        if not reference_bullets:
            continue

        # ROUGE scores
        rouge_scores = scorer.score(summary, reference)
        rouge1_scores.append(rouge_scores["rouge1"].fmeasure)
        rougeL_scores.append(rouge_scores["rougeL"].fmeasure)

        # BERTScore
        P, R, F1 = bert_score(
            [" ".join(generated_bullets)], [reference],
            lang="en", verbose=False
        )
        bert_precisions.append(P[0].item())
        bert_recalls.append(R[0].item())
        bert_f1s.append(F1[0].item())

# --- Final Output ---
print("\n📊 Evaluation Metrics Summary:")
print(f"Avg ROUGE-1:         {np.mean(rouge1_scores):.3f}")
print(f"Avg ROUGE-L:         {np.mean(rougeL_scores):.3f}")
print(f"Avg BERTScore-F1:    {np.mean(bert_f1s):.3f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho


📊 Evaluation Metrics Summary:
Avg ROUGE-1:         0.237
Avg ROUGE-L:         0.154
Avg BERTScore-F1:    0.861


#### Human Comparison: Test Run, Output, and Evaluation

##### Test Run

In [16]:
human_eval_comp_ingredients = "tomatoes, blueberries, chicken, sugar"
human_eval_comp_results = preprocess_ingredient_list_with_health(human_eval_comp_ingredients)

# Run LLM-based summarization on your enriched ingredient data
human_eval_comp_results = enrich_with_health_summaries_v2(human_eval_comp_results, model, tokenizer)

[DEBUG ALIASES 1] before pluralize of tomatoes: set()

[DEBUG] Pluralized 'tomatoes' → 'tomatoe'
[DEBUG ALIASES 2] after pluralize & before scientific additions of tomatoes: {'tomatoe'}

[Model Fallback] tomatoes → Scientific: Solanum lycopersicum
[Model Fallback] Solanum lycopersicum → Common: Tomato
[DEBUG ALIASES 3] after scientific additions of tomatoes: {'Tomato', 'tomatoe', 'Solanum lycopersicum'}

[Query Alias] tomatoes → All aliases: ['tomato', 'tomatoe', 'solanum lycopersicum']
[Google CSE] Retrieved 10 snippets for keyword batch: ['healing', 'vegetarian', 'may lead to']...
[Google CSE] Retrieved 10 snippets for keyword batch: ['supports', 'strengthens', 'alkaline']...
[Google CSE] Retrieved 10 snippets for keyword batch: ['danger', 'supplement', 'vitamin']...
[Google CSE] Retrieved 10 snippets for keyword batch: ['focus', 'not suitable', 'religious restriction']...
[Google CSE] Retrieved 10 snippets for keyword batch: ['toxins', 'healthy', 'mineral']...
[Google CSE] Retrieved

##### Output

In [17]:
# Print trusted API info cleanly
print("TRUSTED API INFO:\n=================")
for entry in human_eval_comp_results:
    print(f"\n🧪 Ingredient: {entry['standard']}")
    print("→ Aliases:", entry["aliases"])

    print("\nTrusted API Info:")
    for source, data in entry["health_info"].items():
        if source == "NER_Snippets":
            continue
        print(f"  • {source}:")
        if isinstance(data, list) and data:
            for item in data:
                if isinstance(item, str):
                    print(f"     - {item}")
                elif isinstance(item, dict):
                    print(f"     - {item.get('title', '')}")
        elif isinstance(data, list):
            print("     - No results")
        else:
            print(f"     - {data}")

# Print the NER Entitiies cleanly
print("\n\nNER ENTITIES:\n=============")
for entry in human_eval_comp_results:
    print(f"\n🧪 Ingredient: {entry['standard']}")
    print("→ Aliases:", entry["aliases"])

    print("\nNER Entities from Web Snippets:")
    if entry["health_info"]["NER_Snippets"]:
        for ent_text, ent_label in entry["health_info"]["NER_Snippets"]:
            print(f"     - {ent_text} ({ent_label})")
    else:
        print("     - No named entities found.")

TRUSTED API INFO:

🧪 Ingredient: tomatoes
→ Aliases: ['tomatoes', 'tomatoe', 'tomato', 'solanum lycopersicum']

Trusted API Info:
  • PubMed:
     - No results
  • OpenFDA:
     - Presence of metal shavings in tomato paste
     - Firm was notified by supplier that specific lots of "expo fresh" Roma tomatoes may be contaminated with Salmonella.
  • RxNorm:
     - 1368361
     - 892499
  • Academic_Articles:
     - Impacts of Soil and Water Conservation Structures on Selected Soil Physicochemical Parameters in Wali Micro-Watershed Ambo District, Central Ethiopia
     - Gibberellins: extending the Green Revolution
     - Resolving spatially distinct phytohormone response zones in Arabidopsis thaliana roots colonized by Fusarium oxysporum
     - Rumen fermentation profile and methane mitigation potential of mango and avocado byproducts as feed ingredients and supplements
     - The impact of asbestos cement pollution in irrigation water on physiological and germination characteristics of T

In [18]:
# Print the results (Summaries) cleanly
print("LLM-GENERATED HEALTH SUMMARIES:\n===============================")
for entry in human_eval_comp_results:
    print(f"\n🧪 Ingredient: {entry['standard']}")
    print("\n💬 LLM-Generated Health Summaries:")
    for category, summary in entry.get("health_summary", {}).items():
        print(f"  [{category.upper()}]: {summary}")
    print("\n")

LLM-GENERATED HEALTH SUMMARIES:

🧪 Ingredient: tomatoes

💬 LLM-Generated Health Summaries:
  [HEALTH EFFECTS]: 
        1. Tomatoes are rich in lycopene, a powerful antioxidant that has been linked to a reduced risk of various types of cancer, including prostate, breast, lung, and stomach cancer (Benefit).
        2. Consuming tomatoes regularly may help improve cardiovascular health by reducing the risk of heart disease and stroke due to their high levels of potassium, vitamin C, and beta-carotene (Benefit).
        3. Tomatoes contain solanine, a glycoalkaloid that can be toxic in large quantities, causing symptoms such as headache, nausea, and paralysis (Concern). However, the amount of solanine in tomatoes is usually low and not a cause for concern when consumed in normal amounts.
  [DIETARY RESTRICTIONS]: 
        1. Tomatoes contain histamine, a compound that can trigger allergic reactions in some individuals, particularly those with mast cell activation disorder or histamine int

##### Reference Summaries

In [19]:
# --- Reference summaries --- (using CHATGPT-4 and human evaluation to fine-tune)
huamn_eval_comp_reference_summaries = {
    "tomatoes-health_effects": (
        "Tomatoes are rich in antioxidants like lycopene, which may help reduce the risk of chronic diseases including cancer and heart disease. "
        "They are a good source of vitamins A and C, supporting skin health and immune function. "
        "Their anti-inflammatory properties may benefit individuals with metabolic syndrome. "
        "Regular tomato consumption is linked to improved cardiovascular health. "
        "They also support eye health due to their lutein and zeaxanthin content."
    ),
    "tomatoes-restrictions": (
        "Tomatoes may cause allergic reactions in some individuals, especially those sensitive to nightshades. "
        "They are acidic and can exacerbate symptoms of acid reflux or GERD. "
        "People with histamine intolerance may experience adverse effects from tomato consumption. "
        "Tomatoes are generally avoided during low-acid or elimination diets."
    ),
    "blueberries-health_effects": (
        "Blueberries are high in antioxidants, particularly anthocyanins, which support brain and cardiovascular health. "
        "They may improve memory and cognitive function, especially in aging populations. "
        "Their anti-inflammatory effects can help reduce oxidative stress and chronic inflammation. "
        "Blueberries are rich in fiber and vitamins C and K. "
        "Regular consumption may improve insulin sensitivity and aid in blood sugar regulation."
    ),
    "blueberries-restrictions": (
        "Blueberries are generally well tolerated, but allergic reactions are possible in rare cases. "
        "They may interact with blood-thinning medications due to their vitamin K content. "
        "People on low-FODMAP diets should moderate intake due to fructose. "
        "Those with salicylate sensitivity may also need to limit blueberry consumption."
    ),
    "chicken-health_effects": (
        "Chicken is a lean protein that supports muscle maintenance and repair. "
        "It contains essential B vitamins, selenium, and phosphorus, which support energy metabolism and bone health. "
        "Skinless chicken is low in saturated fat and may benefit heart health when consumed in moderation. "
        "It can aid in weight management by increasing satiety. "
        "However, excessive consumption of processed or fried chicken may negate these benefits."
    ),
    "chicken-restrictions": (
        "Chicken is not suitable for vegetarians or vegans. "
        "It is forbidden in certain religious diets, such as during Hindu fasting periods or specific Buddhist practices. "
        "People with alpha-gal syndrome (a rare meat allergy) may react to chicken. "
        "Improper handling can lead to foodborne illnesses like Salmonella or Campylobacter infection."
    ),
    "sugar-health_effects": (
        "Excess sugar intake is linked to obesity, type 2 diabetes, and cardiovascular disease. "
        "It contributes to tooth decay and may promote systemic inflammation. "
        "Frequent consumption can cause spikes in blood glucose and insulin levels. "
        "Added sugars provide empty calories with no nutritional benefits. "
        "Some evidence links high sugar consumption to mood swings and fatigue."
    ),
    "sugar-restrictions": (
        "Sugar must be restricted in diabetic and ketogenic diets. "
        "Many religious fasts and clean-eating lifestyles recommend avoiding added sugars. "
        "Individuals with metabolic disorders such as insulin resistance should minimize sugar intake. "
        "People following Whole30 or paleo diets typically eliminate refined sugars entirely."
    ),
}

##### Evaluation

In [20]:
# --- Evaluation functions ---

# Set up metric containers
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

rouge1_scores, rougeL_scores = [], []
bert_precisions, bert_recalls, bert_f1s = [], [], []

# Evaluation loop
for entry in human_eval_comp_results:
    ingredient = entry["standard"].lower()
    for category, summary in entry.get("health_summary", {}).items():
        key = f"{ingredient}-{category.replace(' ', '_')}"
        reference = huamn_eval_comp_reference_summaries.get(key)

        if not reference or not summary.strip():
            continue

        # Extract generated bullets
        generated_bullets = [
            line.strip()[2:].strip()
            for line in summary.splitlines()
            if re.match(r"^\d+\.", line.strip())
        ]
        if not generated_bullets:
            continue

        # Extract reference sentences
        reference_bullets = [s.strip() for s in reference.split('.') if s.strip()]
        if not reference_bullets:
            continue

        # ROUGE scores
        rouge_scores = scorer.score(summary, reference)
        rouge1_scores.append(rouge_scores["rouge1"].fmeasure)
        rougeL_scores.append(rouge_scores["rougeL"].fmeasure)

        # BERTScore
        P, R, F1 = bert_score(
            [" ".join(generated_bullets)], [reference],
            lang="en", verbose=False
        )
        bert_precisions.append(P[0].item())
        bert_recalls.append(R[0].item())
        bert_f1s.append(F1[0].item())

# --- Final Output ---
print("\n📊 Evaluation Metrics Summary:")
print(f"Avg ROUGE-1:         {np.mean(rouge1_scores):.3f}")
print(f"Avg ROUGE-L:         {np.mean(rougeL_scores):.3f}")
print(f"Avg BERTScore-F1:    {np.mean(bert_f1s):.3f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho


📊 Evaluation Metrics Summary:
Avg ROUGE-1:         0.349
Avg ROUGE-L:         0.228
Avg BERTScore-F1:    0.885


In [21]:
ingredient = human_eval_comp_results[3]["standard"].lower()
print(f"\n🧪 Ingredient: {ingredient}")

# --- output just for sugar ---
print("\n📊 Evaluation Metrics Summary:")
print(f"Avg ROUGE-1:         {(rouge1_scores[3]):.3f}")
print(f"Avg ROUGE-L:         {(rougeL_scores[3]):.3f}")
print(f"Avg BERTScore-F1:    {(bert_f1s[3]):.3f}")


🧪 Ingredient: sugar

📊 Evaluation Metrics Summary:
Avg ROUGE-1:         0.294
Avg ROUGE-L:         0.167
Avg BERTScore-F1:    0.875
